# Producer

### Import dependencies

In [1]:
import tweepy
from kafka import KafkaProducer
from json import dumps
import time
import json

### Authentication on Twitter

In [2]:
consumer_key = "dmn51aowr3bYzIWz3ELgARN5x"
consumer_secret = "Q6vHf7FNj1yDvX2vggN6kKLgiDP7t7HL5DNpNvK0akyjzfxTFt"
access_token = "164266780-hcTZEl441WUst50TpiRIZtAd3IHmvPDX5tQUDjqx"
access_token_secret = "AIoEwy9AL52AuHRAKesBph3arofQ4AdLwqYRl8tFnehxd"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

### Connection to Apache Kafka

In [3]:
#Create Kafka Producer

#ip = 'localhost'
ip = '35.228.40.220'
producer = KafkaProducer(bootstrap_servers=[ip+':9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [4]:
#Retrieve TopN Hashtags on the given location

location = 2487956 #san francisco
#location = 2459115 #new york
n = 5 #top n parameter
top_n = [x.get("name") for x in api.trends_place(location)[0].get('trends')[0:n]]
print(top_n)

['Great America', '#PGEshutoff', '#pgepowershutoff', 'Happy Diwali', '#KincadeFire']


### Pipelined Producer

In [6]:
#Define Listener

class PipelineListener(tweepy.StreamListener):
    def __init__(self,tag,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.tag=tag
        super(PipelineListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['tag'] = self.tag
            data['date'] = str(status.created_at).split(" ")[0]#Keep the date, remove time
            data['tweet'] = status.text
            producer.send("DIC",data)
            print(data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch listener in series

time_limit = 100

for _ in range(0,5):
    listeners = []
    streams = []
    for i in range(0,n):
        print(top_n[i])
        listeners.append(PipelineListener(top_n[i],time_limit=time_limit))
        streams.append(tweepy.Stream(auth = api.auth, listener=listeners[i]))
        streams[i].filter(track=[top_n[i]], is_async=False, languages=["en"])
        streams[i].disconnect()

Great America
{'tag': 'Great America', 'date': '2019-10-27', 'tweet': 'RT @BigReidRadio: A lot of speculation, rumors etc.. but I hope everyone who was @ Great America tonight is OK.'}
{'tag': 'Great America', 'date': '2019-10-27', 'tweet': 'RT @adrivnaa47: the parents of the kids + the kids who thought it was funny to yell “shooter” out in Great America should be prosecuted. id…'}
{'tag': 'Great America', 'date': '2019-10-27', 'tweet': 'ya being hot is great and all but has anyone ever told u ur English is so good for not being from America????'}
{'tag': 'Great America', 'date': '2019-10-27', 'tweet': '@2133Dani No shots fired, say police.\n\nhttps://t.co/wjRFtAqBr9'}
{'tag': 'Great America', 'date': '2019-10-27', 'tweet': 'RT @miejaay: ACTIVE SHOOTER AT GREAT AMERICA. ANYONE WHOS HERE BE SAFE.'}
{'tag': 'Great America', 'date': '2019-10-27', 'tweet': 'Great America tonight https://t.co/y0dJv57rSy'}
{'tag': 'Great America', 'date': '2019-10-27', 'tweet': 'A great article by Dennis Pra

{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'Wishes everyone a very happy Diwali~May the Divine light of Diwali bring peace,prosperity,health and Love❤️to your… https://t.co/ITE2LdPcFw'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @SurajRa45940515: #StampsWaliDiwali \n#googlepay #rangolistamp \nWoah! I just got the Rangoli stamp.\nNever been this excited! Thank you GO…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @Arsenal: 💬 "Hi! This is Laca. Enjoy your festivities with your family and friends. Happy Diwali!"\n\n🤳 @LacazetteAlex | #Diwali https://t…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'Happy Diwali to everyone who is celebrating! https://t.co/5WDeygokki'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @chintskap: Whosoever sent this,thank you,it was funny. Happy Diwali. https://t.co/AiRKEEgT7R'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @stevesmith49: Happy Diwali to all of my Indian friends ou

{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @theHMMofficial: Sky full of fireworks, mouth full of sweets. House full of Diyas &amp; heart full of joy. Wishing you all a Very Happy #Diw…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': '@imustbeeddie I wish this was a way of saving our environmental. Happy Diwali btw.'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @abasitpak1: No Eid or even Jumma prayers for occupied Kashmiris. Happy Diwali to India.'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @jeremycorbyn: I would like to wish Hindu, Sikh and Jain communities in the UK and around the world a happy #Diwali.\n\n#Diwali2019 #Deepa…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @Imamofpeace: The below tweet was issued by the biggest hypocrite in the entire geographic region. https://t.co/ItA93wu1kd'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @youwishyouzayn: - happy diwali my beautiful desi moots. hope 

{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @_nxv19_: guys, let’s not be fucking cringe and try move to girls by popping up saying ‘happy diwali’🥴that shits messed'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @BorisJohnson: To everyone celebrating Diwali here in the UK and around the world, I want to wish you all a happy Diwali and a joyful an…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @GeethaArts: #AlaVaikunthapurramuloo family wishes you all a very Happy #Diwali! #DiwaliWithRamulooRamulaa\n\n▶ https://t.co/mi2dUee4ae\n\n@…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'Happy Diwali to all celebrating. Remember being in Calcutta for it a few years ago, amazing experience.'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'Wishing Happy Diwali to you and your family -Manoj Arya'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'May the lights of Diwali diyas fill your home with wealth, happiness, and everything 

{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @ImranTahirSA: Wishing all my brothers and sisters a very happy Diwali'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': "RT @b_yo95: A small light is enough to break darkness. Yet it can't be seen without the other. Happy #Diwali2019 #Diwali everyone~ \nSpark y…"}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': '@rohini_sgh Happy Diwali Mam, you looking so gorgeous in sari.🔥'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @ManUtd: Wishing a very happy #Diwali to all those celebrating today! https://t.co/w5ANVVosBh'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @ChennaiMaheshFC: #SarileruNeekevvaru\nFun on Sets Wait For Worth 😎😎💪\n#MassMb\n#DiwaliMassMBTreat https://t.co/DUKzA2w2Nk'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'Happy Diwali. \nhttps://t.co/ar87PhkWjd\n\nThis festive season, support the free press. Subscribe to Scroll+ now and get 30% off.'}
{'tag': 'Happy Diwa

...

{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'I wish this Diwali the chapter of your life also gets illuminated and you find happiness all around you. \nHappy Diwali! 🙏🙏🙏 @vandanatiwari'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @ThisIsDSP: 😁😁😁👌🏻👌🏻👌🏻 Super Timing All 3 of U sirrr !!!\n\n@AnilRavipudi @actorsubbaraju @vennelakishore https://t.co/ap5NHjowk6'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @AKentsOfficial: Crime Branch Koti &amp; Assistant Kishore Reporting From #SarileruNeekevvaru Sets \n\n@AnilRavipudi @actorsubbaraju @vennelak…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @BorisJohnson: To everyone celebrating Diwali here in the UK and around the world, I want to wish you all a happy Diwali and a joyful an…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @Shritweeets: May this Diwali brings you joy and happiness and I wish all your problems be gone out of your life like rocket whistles ou…'}
{'tag': '

{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @Imamofpeace: The below tweet was issued by the biggest hypocrite in the entire geographic region. https://t.co/ItA93wu1kd'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'Celebrate the festival of lights with #TeamChampions. Wishing you all a happy and prosperous #Diwali #INRCRally… https://t.co/SoJTrMqPu5'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @jeremycorbyn: I would like to wish Hindu, Sikh and Jain communities in the UK and around the world a happy #Diwali.\n\n#Diwali2019 #Deepa…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @jeremycorbyn: I would like to wish Hindu, Sikh and Jain communities in the UK and around the world a happy #Diwali.\n\n#Diwali2019 #Deepa…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @AshiSinghFC1: May the lights of Diwali diyas fill your home with wealth, happiness, and everything that brings you joy! Wish you and yo…'}
{'tag': 'Happy Diwal

{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @SVC_official: Crime Branch Koti &amp; Assistant Kishore Reporting From #SarileruNeekevvaru Sets... \n\n@AnilRavipudi @actorsubbaraju @vennela…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @henrygayle: Wishing you all A Happy Diwali.'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': "RT @iAnkurSingh: Happy Diwali Friends!\n\nA short Message to Liberals saying don't burst crackers. https://t.co/upvggvRHt9"}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'Wishing A Very Happy Diwali 🎊🎉 To You And Your Beautiful Family 😍❤\n\n@iamsrk 🤗🤗🤗🤗🤗🤗🤗 https://t.co/r3Aerl2HPJ'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @AKentsOfficial: Crime Branch Koti &amp; Assistant Kishore Reporting From #SarileruNeekevvaru Sets \n\n@AnilRavipudi @actorsubbaraju @vennelak…'}
{'tag': 'Happy Diwali', 'date': '2019-10-27', 'tweet': 'RT @GWR: Congratulations #Ayodhya on the largest display of oil lamps! Happy Di

{'tag': '#KincadeFire', 'date': '2019-10-27', 'tweet': 'RT @PurpleGimp: Sutter Santa Rosa Regional Hospital has evacuated.  All emergencies go to Santa Rosa Memorial Hospital or Kaiser Santa Rosa…'}
{'tag': '#KincadeFire', 'date': '2019-10-27', 'tweet': '@JohnHipps @juliettekayyem @CNN @RudyGiuliani #kinCADEfire.... CADE, not KAID....'}
{'tag': '#KincadeFire', 'date': '2019-10-27', 'tweet': 'RT @805Weather: Time lapse of the last hour on the #KincadeFire as conditions continue to deteriorate... #CAwx #CAfire #CAwind #PSPS #RFW 🚩…'}
{'tag': '#KincadeFire', 'date': '2019-10-27', 'tweet': 'RT @thejdmorris: This is a map of Sonoma County (pop. 500,000), one of the best places on Earth.\n\nAlmost all of it is on fire, under evacua…'}
{'tag': '#KincadeFire', 'date': '2019-10-27', 'tweet': 'RT @CAFireScanner: Looks like 15 strike teams of engines  (75 engines) already ordered from Oregon, responding in on Sunday. May not all en…'}
{'tag': '#KincadeFire', 'date': '2019-10-27', 'tweet': 'RT @Ro

KeyboardInterrupt: 

### Parallel Producer

In [ ]:
#Define Listener

class ParallelListener(tweepy.StreamListener):
    def __init__(self,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        super(ParallelListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['date'] = str(status.created_at)
            data['tweet'] = status.text
            json_data = json.dumps(data)
            producer.send("test",json_data)
            print(json_data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch all listener together

listener = ParallelListener(time_limit=1000)
myStream = tweepy.Stream(auth = api.auth, listener=listener)
myStream.filter(track=top_n, is_async=False, languages=["en"])